In [22]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
img1 = cv2.imread(r"D:\Project\PyProject\Ai\opencv\data\building\building1.jpg",0)
img2 = cv2.imread(r"D:\Project\PyProject\Ai\opencv\data\building\building2.jpg",0)

In [20]:
def cv_show(name,img):
    cv2.imshow(name,img)
    while True:
        if cv2.waitKey(30) == 113:
            break
    cv2.destroyAllWindows()
def cv_show3(img1,img2,img3):
    cv2.imshow("img1",img1)
    cv2.imshow("img2",img2)
    cv2.imshow("img3",img3)
    while True:
        if cv2.waitKey(30) == 113:
            break
    cv2.destroyAllWindows()
    
    
    

In [6]:
cv_show("img1",img1)

In [7]:
cv_show("img2",img2)

In [8]:
# 建立SIFT生成器
sift = cv2.xfeatures2d_SURF.create()  # cv2.xfeatures2d_SURF.create() opencv-python3.4.2.16

In [9]:
#  检测SIFT特征点,并计算描述子
kp1,des1 = sift.detectAndCompute(img1,None)
kp2,des2 = sift.detectAndCompute(img2,None)

In [10]:
#  将结果转换成NumPy数组
nkps1 = np.float32([kp.pt for kp in kp1])
nkps2 = np.float32([kp.pt for kp in kp2])

In [11]:
print(nkps1)

[[379.03317 324.22183]
 [624.23846 312.819  ]
 [380.4851  324.25296]
 ...
 [400.15558 230.291  ]
 [296.8988  196.32121]
 [298.12024 269.03864]]


In [12]:
ratio=0.75
reprojThresh=4.0

In [13]:
#  匹配两张图的所有特征点,返回匹配结果
# M = self.matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh)
# 建立暴力匹配器
matcher = cv2.BFMatcher()
# 使用KNN 检测来自A,B 图的SIFT特征匹配对比, K = 2
rawMatches = matcher.knnMatch(des1, des2, 2)

In [14]:
matches = []
for m in rawMatches:
    #  当最近距离跟次近的比值小于ratio值时,保留此匹配对:
    if len(m) == 2 and m[0].distance < m[1].distance * ratio:
        #  存储两个点在featuresA , featuresB 中的索引
        matches.append((m[0].trainIdx,m[0].queryIdx))
if len(matches) > 4:
    #  获取匹配对的点坐标
    ptsA = np.float32([nkps1[i] for (_, i) in matches])
    ptsB = np.float32([nkps2[i] for (i, _) in matches])

    #  计算视角变换矩阵
    (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)

    #  返回结果
#     return (matches, H, status)

In [15]:
print(H)

[[ 1.25669485e+00 -9.75047121e-02 -3.12882121e+02]
 [ 1.93295650e-01  1.16304969e+00 -5.64514359e+01]
 [ 4.44493077e-04 -5.98479614e-05  1.00000000e+00]]


In [16]:
#  将图片A进行视角变换,result是变换后的图片
result = cv2.warpPerspective(img1, H, (img1.shape[1] + img2.shape[1], img2.shape[0]))

In [17]:
cv_show("result",result)

In [18]:
#  将图片B传入result图片最左端
result[0:img2.shape[0], 0:img2.shape[1]] = img2

In [21]:
cv_show3(img1,img2,result)

In [23]:
gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)



error: OpenCV(3.4.2) c:\projects\opencv-python\opencv\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'
